In [1]:
# Importing Libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Variables 
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Loading Data
train_dataset = torchvision.datasets.MNIST(root='./data', 
        train = True, 
        transform=transforms.Compose([
          transforms.Resize((32, 32)),
          transforms.ToTensor(),
          transforms.Normalize(mean=(0.1307,), std=(0.3081,))
        ]), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', 
        train = False, 
        transform=transforms.Compose([
          transforms.Resize((32, 32)),
          transforms.ToTensor(),
          transforms.Normalize(mean=(0.1307,), std=(0.3081,))
        ]), download=True)

train_loader = torch.utils.data.DataLoader(
  dataset = train_dataset,
  batch_size=batch_size,
  shuffle=True)
test_loader = torch.utils.data.DataLoader(
  dataset = test_dataset,
  batch_size=batch_size,
  shuffle=True)

In [8]:
# LeNet5 Model

class LeNet5(nn.Module):
  def __init__(self, num_classes):
    super(LeNet5, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
      nn.BatchNorm2d(6),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(400, 120)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(120, 84)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(84, num_classes)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    out = self.relu(out)
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

In [9]:
# Setting HuperParameters
model = LeNet5(num_classes).to(device)

# Loss Function
cost = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

total_step = len(train_loader)

In [12]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = cost(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 400 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1437
Epoch [1/10], Step [800/938], Loss: 0.0239
Epoch [2/10], Step [400/938], Loss: 0.0736
Epoch [2/10], Step [800/938], Loss: 0.0261
Epoch [3/10], Step [400/938], Loss: 0.0046
Epoch [3/10], Step [800/938], Loss: 0.0181
Epoch [4/10], Step [400/938], Loss: 0.1688
Epoch [4/10], Step [800/938], Loss: 0.1086
Epoch [5/10], Step [400/938], Loss: 0.0079
Epoch [5/10], Step [800/938], Loss: 0.0424
Epoch [6/10], Step [400/938], Loss: 0.0027
Epoch [6/10], Step [800/938], Loss: 0.0016
Epoch [7/10], Step [400/938], Loss: 0.0020
Epoch [7/10], Step [800/938], Loss: 0.0255
Epoch [8/10], Step [400/938], Loss: 0.0043
Epoch [8/10], Step [800/938], Loss: 0.0023
Epoch [9/10], Step [400/938], Loss: 0.0010
Epoch [9/10], Step [800/938], Loss: 0.0109
Epoch [10/10], Step [400/938], Loss: 0.0035
Epoch [10/10], Step [800/938], Loss: 0.0071


In [13]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.83 %
